In [2]:
# %load "classes/main.py"
import osmnx as ox
import pandas as pd
from osmnx import settings, graph, geocoder
from shapely import Polygon
import time
import numpy as np
settings.max_query_area_size = 15000000000000

class Osm_Wrapper():

    '''
    Class to read OSM data and respond back with outputs

    Author: Abhas Dudeja

    Dependencies: Pandas, GeoPandas, OSMnx, Shapely.
    Date: 04 01 2024
    '''

    @staticmethod
    def get_bounds(name,type="country"):
        '''
        Return bounds of any name value, default type of name "country"
        '''
        res = geocoder.geocode_to_gdf({type:name})
        return res.unary_union.bounds
    
    # @staticmethod
    # def get_cities(bbox,count=10):
        

In [3]:
city = 'Bengaluru'
country = "India"
name = f'{city}, {country}'

In [4]:
# set bounding blox using country name
bbox = Osm_Wrapper.get_bounds(country,"country")

In [5]:
country_polygon = ox.geocode_to_gdf(country).geometry[0]
# country_polygon = Polygon(country_polygon.geometry)

In [6]:
cities = ox.features_from_polygon(polygon = country_polygon, tags={"place": "city"})
cities = pd.DataFrame(cities,columns=cities.columns).drop('geometry', axis = 1)

In [ ]:
cust_filter = '["route"="bus"]'

In [ ]:
def download_public_transport_networks(city_names):
    """Downloads public transport networks for a list of cities."""

    for city_name in city_names:
        try:
            city_name = str(city_name)
            print(f"Downloading network for {city_name}...")
            G = ox.graph_from_place(city_name, network_type='all')
            ox.save_graph_geopackage(G,filename=f"{city_name}_public_transport_network")
            print(f"Network for {city_name} downloaded successfully.")

            # Consider rate limiting to avoid overwhelming servers
            time.sleep(0.5)  # Adjust delay as needed

        except Exception as e:
            print(f"Error occurred while downloading network for {city_name}: {e}")


In [ ]:
def get_cities_from_osm(num_cities=5):
    """Retrieves city names from OSM within a specified bounding box,
    leveraging place boundary geometries for accuracy.
    """
    # set bounding blox using country name
    bbox = get_country_bounds(country)

    try:
        # Retrieve place geometries for more accurate city identification
        places_gdf = ox.features_from_bbox(north=bbox[0],south=bbox[1],east=bbox[2],west=bbox[3], tags={'place': 'city'})
        
        cities_gdf = places_gdf.sort_values('population', ascending=False).head(num_cities)
        print(places_gdf)
        city_names = list(cities_gdf['name'])
        
        return city_names

    except Exception as e:
        print(f"Error occurred while retrieving cities: {e}")
        return []

In [ ]:
city_names = get_cities_from_osm(num_cities=5)

In [ ]:
# Download public transport networks
# download_public_transport_networks(city_names)